In [6]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip -q install langchain huggingface_hub openai tiktoken
!pip -q install chromadb duckduckgo-search

In [ ]:
!pip show langchain

# LangChain Expression Language


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.schema.output_parser import StrOutputParser

In [ ]:
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
    )

model2 = OpenAI(
    model="text-davinci-003",
    temperature=0
    )

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me an intersting fact about {subject}"
    )

In [ ]:
chain = prompt | model

In [ ]:
chain.invoke({"subject": "Elvis"})

In [ ]:
chain = prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"subject": "Elvis"})

In [ ]:
chain = prompt | model2 | StrOutputParser()

In [ ]:
chain.invoke({"subject": "Elvis"})

## Bindings

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me 3 intersting facts about {subject}"
    )

In [ ]:
chain = prompt | model.bind(stop=["\n"]) | StrOutputParser()


In [ ]:
chain.invoke({"subject": "Elvis"})

## Adding OpenAI Functions

In [ ]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]
functions_chain = prompt | model.bind(function_call= {"name": "joke"}, functions= functions)

In [ ]:
functions_chain.invoke({"subject": "bears"}, config={})

### Functions Output Parser


In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

functions_chain = (
    prompt
    | model.bind(function_call= {"name": "joke"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [ ]:
response = functions_chain.invoke({"subject": "bears"})

response

In [ ]:
response['punchline']

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

functions_chain = (
    prompt
    | model.bind(function_call= {"name": "joke"}, functions= functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [ ]:
functions_chain.invoke({"subject": "bears"})

## Retrievers

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
# Create the retriever
fake_docs = ["James bond works for MI6","Bond is a spy",
             "James Bond has a licence to kill", "James Bond likes cats"]
vectorstore = Chroma.from_texts(fake_docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("Who is James Bond?")

In [ ]:
chain.invoke("What does James Bond like to do?")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"question": "where does James work?", "language": "english"})

In [ ]:
chain.invoke({"question": "where does James work?", "language": "italian"})

## Tools

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

In [ ]:
search = DuckDuckGoSearchRun()

In [ ]:
template = """turn the following user input into a search query for a search engine:

{input}"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = prompt | model | StrOutputParser() | search

In [ ]:
chain.invoke({"input": "Who played james bond first"})

In [ ]:
chain = prompt | model | StrOutputParser()
chain.invoke({"input": "Who played james bond last"})

## Arbitary Functions

In [ ]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt | model

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | model

In [ ]:
chain.invoke({"foo": "bars", "bar": "gahs"})